# lcel 인터페이스

- 저자 : [Jeonggi Park] (https://github.com/jeongkpa)
- 동료 검토 : [Yookyung Jeon] (https://github.com/sirena1), [Wooseok Jeong] (https://github.com/jeong-wooseok)
- 교정 : [q011] (https://github.com/q0211)
-이것은 [Langchain Open Tutorial]의 일부입니다.

[!
[! [열린github] (https://img.shields.io/badge/open%20in%20github-181717?style=flat-square&logo=github&logocolor=White)한

## 개요

Langchain Expression Language (LCEN)는 Langchain에서 사용자 정의 체인의 생성 및 관리를 단순화하도록 설계된 강력한 인터페이스입니다.
언어 모델 체인을 구축하고 실행하는 표준화 된 방법을 제공하는 런닝 가능한 프로토콜을 구현합니다.


### 목차

- [개요] (#개요)
- [환경 설정] (#Environment-Setup)
-[lcel runnable 프로토콜] (#lcel-runnable-protocol)
-[스트림 : 실시간 출력] (#stream-real-time output)
- [호출] (#호출)
-[배치 : 단위 실행] (#Batch-Unit-Excution)
- [비동기 스트림] (#Async-stream)
- [Async invoke] (#Async-Invoke)
- [비동기 배치] (#Async-Batch)
- [병렬] (#평행)
-[배치의 병렬성] (#병렬 처리)

### 참조

- [Langsmith Doc] (https://docs.smith.langchain.com/)
---

## 환경 설정

환경을 설정하십시오.자세한 내용은 [환경 설정] (https://wikidocs.net/257836)을 참조하십시오.

**[메모]**
-``Langchain-Opentutorial '' '는 사용하기 쉬운 환경 설정 세트, 유용한 기능 및 튜토리얼을위한 유틸리티를 제공하는 패키지입니다.
-자세한 내용은 [``````langchain-opentutorial``] (https://github.com/langchain-opentutorial/langchain-opentutorial-pypi)를 확인할 수 있습니다.

세트 환경 변수는 .env입니다.

.env_sample의 내용을 복사하여 설정 한 키로 .env에로드하십시오.

In [13]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [9]:
%%capture --no-stderr
!pip install langchain-opentutorial


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
#필요한 패키지를 설치하십시오
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain-openai",
        "langchain",
        "python-dotenv",
        "langchain-core",
    ],
    verbose=False,
    upgrade=False,
)

In [11]:
#환경 변수를 설정합니다
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "<Your OpenAI API KEY>",
        "LANGCHAIN_API_KEY": "<Your LangChain API KEY>",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LangSmith-Tracking-Setup",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


## lcel 실행 가능한 프로토콜

---

사용자 정의 체인을 가능한 한 쉽게 만들기 위해````runnable '' '프로토콜을 구현했습니다.

````runnable '' '프로토콜은 대부분의 구성 요소에서 구현됩니다.

사용자 정의 체인을 쉽게 정의하고 표준 방식으로 호출 할 수있는 표준 인터페이스입니다.표준 인터페이스에는 포함됩니다

-```stream '' ': 응답의 덩어리를 스트리밍합니다.
-```````````` ': 입력에서 체인을 호출하십시오.
-``배치 '': 입력 목록에 체인을 호출합니다.

비동기 방법도 있습니다

-````astream '' ': 응답의 덩어리가 비동기식으로 스트림 덩어리.
-````ainvoke '' ': 입력에서 체인을 비동기로 호출하십시오.
-````Abatch '' ': 입력 목록에 대해 체인을 비동기로 호출합니다.
-````astream_log '' ': 최종 응답과 중간 단계가 발생할 때 스트리밍됩니다.

### 추적을 기록하십시오

우리는 Langsmith에 추적을 기록하는 여러 가지 방법을 제공합니다.아래에서 추적 가능한 ()를 사용하는 방법을 강조하겠습니다.

아래 코드를 사용하여 Langsmith에서 흔적을 기록하십시오.

In [14]:
import openai
from langsmith import wrappers, traceable

#Auto-Trace LLM은 텍스트 내에서 호출됩니다
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
#외부 : 안녕하세요!오늘 당신을 어떻게 도와 드릴까요?


'Hello! How can I assist you today?'

lcel 구문을 사용하여 체인을 만듭니다.

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

#Chatopenai 모델을 인스턴스화하십시오.
model = ChatOpenAI()
#주어진 주제에 대한 농담을 요구하는 프롬프트 템플릿을 만듭니다.
prompt = PromptTemplate.from_template("Describe the {topic} in 3 sentences.")
#프롬프트와 모델을 연결하여 대화 체인을 만듭니다.
chain = prompt | model | StrOutputParser()

## 스트림 : 실시간 출력

이 기능은````chain.stream``` 메소드를 사용하여 주어진 주제에 대한 데이터 스트림을 생성하고,이를 반복하고, 각 데이터의````content '' '를 즉시 출력합니다.
```end = ""«인수는 출력 후 신자 라인을 비활성화하며,``flush = true ''인수는 출력 버퍼를 즉시 비우게합니다.

In [16]:
#chain.stream 메소드를 사용하여 주어진 주제에 대한 데이터 스트림을 만들어 반복하고 각 데이터의 내용을 즉시 출력하십시오.
for token in chain.stream({"topic": "multimodal"}):
    #최신 데이터의 컨텐츠를 출력하십시오.
    print(token, end="", flush=True)

#예제 출력
#멀티 모달 접근법은 학습과 이해를 향상시키기 위해 시각, 청각 및 운동학과 같은 여러 커뮤니케이션 모드를 사용하는 것입니다.다른 감각 입력을 통합함으로써 학습자는보다 전체적이고 몰입감있는 방식으로 자료에 참여할 수 있습니다.이 접근법은 특히 다양한 학습 스타일과 선호도를 충족하는 데 효과적입니다.


The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to enhance learning and understanding. By incorporating various modalities, individuals can access information in ways that cater to their unique learning styles and preferences. This approach allows for a more comprehensive and inclusive learning experience that promotes deeper engagement and retention of information.

## 부르다

`````````````````````` ''객체는 주제를 인수로 받아들이고 해당 주제에 대한 처리를 수행합니다.

In [17]:
#'chatgpt'주제와 함께 사전을 전달하고 체인 객체의 호출 방법을 호출하십시오.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT is a state-of-the-art conversational AI model developed by OpenAI that can engage in natural and coherent conversations with users. It uses the GPT-3 architecture, which allows it to generate human-like responses based on the input it receives. ChatGPT is known for its ability to understand context, provide relevant information, and maintain engaging conversations across a wide range of topics.'

## 배치 : 단위 실행

함수```chain.batch '«는 여러 사전을 포함하는 목록을 인수로 받아들이고 각 사전의``주제' '키의 값을 사용하여 배치 처리를 수행합니다.

In [18]:
#주어진 주제 목록을 배치 프로세스로 함수를 호출하십시오.
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT is an advanced conversational AI model developed by OpenAI that can engage in natural and engaging conversations with users. It is trained on a diverse range of internet text data to understand and generate human-like responses. ChatGPT is capable of providing information, answering questions, and engaging in meaningful dialogues with users on a variety of topics.',
 'Instagram is a social media platform where users can share photos and videos with their followers. The app allows users to edit and enhance their photos with filters and captions before posting. Users can also interact with others by liking, commenting, and messaging on their posts.']

````max_concurrency '' ''매개 변수를 사용하여 동시 요청 수를 설정할 수 있습니다.

``config ''사전은```max_concurrency ''키를 사용하여 동시에 처리 할 수있는 최대 작업 수를 설정합니다.여기에서는 최대 3 개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [19]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "multimodal"},
        {"topic": "programming"},
        {"topic": "machineLearning"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT is an AI-powered chatbot developed by OpenAI that can engage in text-based conversations with users. It uses a machine learning model trained on a diverse range of internet text to generate responses that are contextually relevant and engaging. ChatGPT can provide information, answer questions, or just have a casual conversation with users.',
 'Instagram is a social media platform where users can share photos and videos with their followers. It allows users to edit and enhance their photos with filters and editing tools. Users can also engage with others by liking, commenting, and sharing posts.',
 'The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to enhance learning and understanding. This approach recognizes that individuals learn in different ways and can benefit from a variety of sensory experiences. By incorporating different modes of communication, educators can cater to diverse learning styles and create

## 비동기 스트림

```chain.astream ''함수는 비동기 스트림을 생성하고 주어진 주제에 대한 메시지를 비동기로 처리합니다.

스트림에서 메시지의 순차적으로 메시지를 순차적으로 수신하기 위해 루프에 비동기식을 사용하고 메시지의 내용을 즉시 인쇄하는 인쇄 기능 (```s.content``)를 인쇄합니다.```end = "««인쇄 후 라인 래핑을 비활성화하고,``flush = true '' '는 출력 버퍼를 비워 지도록 즉각적인 인쇄를 보장합니다.

In [20]:
#비동기 스트림을 사용하여 'YouTube'주제에서 메시지를 처리하십시오.
async for token in chain.astream({"topic": "YouTube"}):
    #메시지 내용을 인쇄하십시오.신축성없이 직접 출력하고 버퍼를 비 웁니다.
    print(token, end="", flush=True)

YouTube is a video-sharing platform where users can upload, view, and interact with videos on a wide range of topics. It has become one of the largest and most popular websites on the internet, with millions of users visiting daily. Users can subscribe to channels, like and comment on videos, and create playlists to save their favorite content.

## Async 호출

````ainvoke '' '`````````' '객체는 주어진 인수와 비동기 적으로 작동을 수행합니다.여기서, 우리는``````topic '' '이라는 키로 사전을 전달하고 있습니다.```nvda``` (nvidia's ticker)라는 값을 논쟁으로 사용합니다.이 방법을 사용하여 특정 주제에 대한 비동기 적으로 처리를 요청할 수 있습니다.

In [21]:
#비동기 체인 객체의 'Ainvoke'메소드를 호출하여 'NVDA'주제를 처리하십시오.
my_process = chain.ainvoke({"topic": "NVDA"})

In [22]:
#비동기 프로세스가 완료 될 때까지 기다리십시오.
await my_process

'The National Visa Center (NVDA) is a government agency responsible for processing immigrant visa applications for individuals seeking to live permanently in the United States. They review and approve the necessary documentation, schedule interviews at U.S. embassies or consulates, and ensure that applicants meet all eligibility requirements for a visa. The NVDA plays a crucial role in managing the visa application process and facilitating legal immigration to the United States.'

## 비동기 배치

함수```Abatch ''는 일련의 동작을 비동기 적으로 배치합니다.

이 예에서는```````````````````````````````````````````````` '`````Topic' '``````Topic' '이````Abath' '````Abatch' '````````````' '가 사용됩니다.

```Await ''키워드는 이러한 비동기 작업이 완료되기를 기다리는 데 사용됩니다.

In [23]:
#주어진 주제에서 비동기 배치 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [24]:
#비동기 배치 프로세스가 완료 될 때까지 기다리십시오.
await my_abatch_process

['YouTube is a popular video-sharing platform where users can upload, view, and share videos on a wide range of topics. It has become a major source of entertainment, education, and information for millions of people around the world. With features like live streaming, monetization options, and a vast library of content, YouTube has revolutionized the way we consume video content.',
 'Instagram is a popular social media platform where users can share photos and videos with their followers. It allows for creative expression through filters, captions, and hashtags. Users can also interact with others by liking, commenting, and messaging on posts.',
 'Facebook is a popular social media platform where users can connect with friends and family, share photos and updates, and join groups based on their interests. It was founded by Mark Zuckerberg in 2004 and has since grown to have billions of active users worldwide. Users can also follow pages and businesses to stay updated on news and event

## 평행한

Langchain 표현 언어가 병렬 요청을 어떻게 지원하는지 살펴 보겠습니다.예를 들어,````runnableparallel '' ''(종종 사전 형식으로 작성)를 사용하면 각 요소를 병렬로 실행합니다.

다음은``langchain_core.runnables`````` 모듈에서````runnableparallel '' '등급을 사용하여 두 작업을 병렬로 실행하는 예입니다.

```````````````````````````````````````````````````````````)``````````````````````` ''가 주어진````country '' '에 대한 자본과 영역을 얻으십시오.

이 체인은 각각``모델 '' '및 파이프 (```|`````````' '연산자를 통해 연결됩니다.마지막으로, 우리는``runnableparallel``` 클래스를 사용 하여이 두 체인을 키```자본 ''````````````` ''`````````` ''를 결합하여 평행하게 실행할 수있는 객체를 만듭니다.

In [25]:
from langchain_core.runnables import RunnableParallel

#{country}의 자본을 요구하는 체인을 만듭니다.
chain1 = (
    PromptTemplate.from_template("What is the capital of {country}?")
    | model
    | StrOutputParser()
)

#{country}의 영역을 요구하는 체인을 만듭니다.
chain2 = (
    PromptTemplate.from_template("What is the area of {country}?")
    | model
    | StrOutputParser()
)

#위의 두 체인을 병렬로 생성하는 병렬 실행 체인을 만듭니다.
combined = RunnableParallel(capital=chain1, area=chain2)

``«chain1.invoke ()`````````````````````````````````` ''가 호출됩니다.

논쟁으로, 그것은``캐나다 ''``````country '' '라는 값을 가진 사전을 통과합니다.

In [26]:
#실행 체인 1.
chain1.invoke({"country": "Canada"})

'Ottawa'

``«chain2.invoke ()```````` ', 이번에는 다른 나라를지나 국가 키를 위해``미국' ''을 통과하십시오.

In [27]:
#실행 체인 2.
chain2.invoke({"country": "USA"})

'The total land area of the United States is approximately 3.8 million square miles (9.8 million square kilometers).'

``invoke '' '``````결합' '객체는 주어진``컨트리' '에 대한 프로세싱을 수행합니다.

이 예에서``USA ''주제는```````````````` ''메소드로 전달됩니다.

In [28]:
#병렬 실행 체인을 실행하십시오.
combined.invoke({"country": "USA"})

{'capital': 'The capital of the United States of America is Washington, D.C.',
 'area': 'The total area of the United States is approximately 9.8 million square kilometers (3.8 million square miles).'}

## 배치의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합 될 수 있습니다.배치와 병렬 처리를 사용해 보겠습니다.

````chain1.batch ''함수는 여러 사전을 포함하는 목록을 인수로 취하고 각 사전의 "주제"키에 해당하는 값을 처리합니다.이 예에서는 두 가지 주제 인 "Canada"와 "United States"를 배치하고 있습니다.

In [29]:
#배치 처리를 수행하십시오.
chain1.batch([{"country": "Canada"}, {"country": "USA"}])


['Ottawa', 'The capital of the United States of America is Washington, D.C.']

````chain2.batch '' '``기능은 여러 사전을 목록으로 사용하고 배치 처리를 수행합니다.

이 예에서, 우리는``캐나다 ''와``미국 ''등 두 국가의 처리를 요청합니다.

In [30]:
#배치 처리를 수행하십시오.
chain2.batch([{"country": "Canada"}, {"country": "USA"}])


['The total land area of Canada is approximately 9.98 million square kilometers (3.85 million square miles), making it the second largest country in the world by land area.',
 'The total land area of the United States is approximately 3.8 million square miles.']

결합 된 배치 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다.

이 예에서는 두 국가, 캐나다와 미국의 인수 및 배치 데이터로 두 개의 사전 객체를 포함하는 목록을 가져옵니다.

In [31]:
#주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "Canada"}, {"country": "USA"}])


[{'capital': 'Ottawa',
  'area': 'The total area of Canada is approximately 9.98 million square kilometers (3.85 million square miles), making it the second-largest country in the world by land area.'},
 {'capital': 'The capital of the United States of America is Washington, D.C.',
  'area': 'The total land area of the United States is approximately 3.8 million square miles (9.8 million square kilometers).'}]